<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project: Harmony
## 3.3 Recommending Colours and Colour Palettes with Llama3
> Authors: Eugene Matthew Cheong
---

## Table of Contents ##

#### 1. Web Scraping

- [1.1 Scraping Lian Seng Hin Website](1.1_web_scraping_liansenghin.ipynb)
- [1.2 Scraping Hafary Website](1.2_web_scraping_hafary.ipynb)
- [1.3 Scraping Lamitak Website](1.3_web_scraping_lamitak.ipynb)
- [1.4 Scraping Nippon Website](1.4_web_scraping_nippon.ipynb)
- [1.5 Consolidate All Product Database](1.5_consolidate_product_database.ipynb)

#### 2. Preprocessing

- [2.1 Processing Canva Palettes](2.1_processing_canva_palette.ipynb)

#### 3. Modelling

- [3.1 Matching Input Photo to Products](3.1_matching_input_photo_to_products.ipynb)
- [3.2 Recommending Canva Palette to Products](3.2_recommending_canva_palette_to_product.ipynb)
- [3.3 Recommending Colours and Colour Palettes with Llama3](3.3_recommending_colours_and_colour_palettes_with_llama3.ipynb)

---

# Import Modules

In [10]:
import re
import os
import pandas as pd
import numpy as np


import ollama

In [13]:
file_path = '../datasets/color_palette_df.csv'
canva_palettes_df = pd.read_csv(file_path)

### Formatting the colour palette dataframe into a string for the LLM to reference

In [14]:
# Create a formatted string for each row to display palette name and the four colors
formatted_strings = ["'Palette: {} - Colors: {}, {}, {}, {}'".format(row['Name'], row['Color 1'], row['Color 2'], row['Color 3'], row['Color 4']) for index, row in canva_palettes_df.iterrows()]

# Join all formatted strings into a single string with each palette on a new line
color_palette_output = "Only recommend colour palettes from this list:\n"

color_palette_list_string = "\n".join(formatted_strings)

color_palette_output += color_palette_list_string

In [15]:
print(color_palette_output)

Only recommend colour palettes from this list:
'Palette: Rosettes and Cream - Colors: #EF7C8E, #FAE8E0, #B6E2D3, #D8A7B1'
'Palette: Rosy Flamingo - Colors: #E8B4B8, #EED6D3, #A49393, #67595E'
'Palette: Summer Splash - Colors: #05445E, #189AB4, #75E6DA, #D4F1F4'
'Palette: Pastel Dreams - Colors: #FBE7C6, #B4F8C8, #A0E7E5, #FFAEBC'
'Palette: Room for Comfort - Colors: #E7D2CC, #B9B7BD, #868B8E, #EEEDE7'
'Palette: The Deep Blue - Colors: #050A30, #000C66, #0000FF, #7EC8E3'
'Palette: Emerald Entrance - Colors: #B68D40, #F4EBD0, #122620, #D6AD60'
'Palette: Mermaid Lagoon - Colors: #145DA0, #0C2D48, #2E8BC0, #B1D4E0'
'Palette: Retro Punch - Colors: #2FF3E0, #F8D210, #FA26A0, #F51720'
'Palette: Healthy Leaves - Colors: #3D550C, #81B622, #ECF87F, #59981A'
'Palette: Right on Time - Colors: #F6EEE0, #E4B7A0, #A45C40, #C38370'
'Palette: Window Tide - Colors: #41729F, #5885AF, #274472, #C3E0E5'
'Palette: Padlocked Doors - Colors: #B99095, #FCB5AC, #B5E5CF, #3D5B59'
'Palette: Salmon Sushi - Colors:

# Initiating LLAMA3
---
#### Reminder: Remember to run "ollama serve" in the terminal before running this script

### Limitations

There are quite a number of limitations when using Ollama. One of them is that I am unable to set the temperature or the tone. Another is that it does not do well with looking up on the history of the chat. So I had to add reminders like only using the palettes provided, which sometimes does not always work. Probably have to look somewhere else to have more control.

### Gathering Hex codes and recognized Colour Palettes

After testing, the LLM is able to recommend hex codes and colour palettes which is pretty handy, especially for colour palettes that are not in the dataframe. I will be using that to generate more custom colour palettes and add to the dataframe, so that the user has more variety to reccommend the clients.

In [ ]:
#Test prompt
# I have a 4 year old daughter who loves the colour pink. Could you suggest colors that would be suitable for her?

In [ ]:
def extract_hex_codes(text):
    # Regex pattern to match hex codes
    pattern = r'#[0-9A-Fa-f]{3}(?:[0-9A-Fa-f]{3})?\b'
    
    # Find all matches of the pattern in the text
    hex_codes = re.findall(pattern, text)
    
    return hex_codes

In [ ]:
def match_palette_names(response, df):
    matched_names = []

    # Using regex to also include common conjunctions and prepositions like 'and', 'or', 'of', etc.
    potential_names = re.findall(r'\b[A-Z][a-z]+(?:\s(?:and|or|of)\s[A-Z][a-z]+)*', response)
    
    # Expand the search to include more potential matches by allowing the inclusion of multiple segments with conjunctions or prepositions
    extended_search = re.findall(r'\b[A-Z][a-z]+(?:\s(?:and|or|of)\s[A-Z][a-z]+|\s[A-Z][a-z]+)*', response)
    potential_names.extend(extended_search)
    
    # Check each potential name against the palette names in the DataFrame
    for name in set(potential_names):  # Use set to avoid duplicates from extended search
        if name in df['Name'].values:
            matched_names.append(name)
    
    return matched_names

### Setting the prompt for the introduction and conditions for LLM to follow

In [ ]:
introduction_input = "You are an interior designer in this app and are tasked to suggest colour palettes to suit their preferences based on their description. Set a professional tone when responding."

condition_input = "When suggesting colours, do include the hex codes in all of the response and have it as a seperate list. Do not suggest any products or furnitures. Only suggest colours and colour palettes."

### Putting the introduction, condition and colour palettes reference list into the conversation history

In [ ]:
# Initialize conversation history
conversation_history = [
    {'role': 'system', 'content': introduction_input},
    {'role': 'system', 'content': condition_input},
    {'role': 'system', 'content': color_palette_output}
]

### Running LLAMA3

Only stops when user's input is empty

In [ ]:
hex_codes_list = []
colour_palette_list = []

In [20]:
while True:
    user_input = input("You: ")

    if not user_input:
        break
    
    # Add the new user input to the conversation
    conversation_history.append({'role': 'system', 'content': condition_input})
    conversation_history.append({'role': 'system', 'content': color_palette_output})
    conversation_history.append({'role': 'user', 'content': user_input})
    
    # Generate response
    stream = ollama.chat(
        model='llama3',
        messages=conversation_history,
        stream=True
    )
    
    response = ""
    for chunk in stream:
        print(chunk['message']['content'], end='', flush=True)
        response += chunk['message']['content']

    
    # Add the response to the conversation history
    conversation_history.append({'role': 'system', 'content': response})

    #This will extract all hex codes the LLM generate
    codes = extract_hex_codes(response)
    for code in codes:
        if code not in hex_codes_list:
            hex_codes_list.append(code)
            

    #This will extract all colour palettes the LLM generate that matches with colour palettes in the Dataframe
    matched_names = match_palette_names(response, canva_palettes_df)
    if matched_names:
        for name in matched_names:
            if name not in colour_palette_list:
                colour_palette_list.append(name)

What a sweetie! I'd be happy to help you find some lovely color palettes for your little princess!

Based on your request, I'll recommend some pink-inspired palettes from my list:

1. **Lilac Shimmer**: A soft and romantic palette featuring #3E004A (a light lilac), #ADE292 (a gentle yellow-green), #4120A9 (a deep berry pink), and #7800A2 (a rich plum). Perfect for a sweet 4-year-old!
2. **Candy Style**: A playful and whimsical palette with #5558D8 (a pastel blue-pink), #D4BBDD (a soft peach), #FBD3DA (a warm beige), and #8ADFE3 (a pale aqua). Your little one might enjoy these colors!
3. **Fresh Succulent**: A bright and cheerful palette featuring #13292A (a deep green), #91CABE (a vibrant pink), #499FA4 (a soft peach), and #B5DFD6 (a creamy white). This palette is perfect for a lively and energetic 4-year-old!
4. **Blush of Trees**: A gentle and soothing palette with #E5A298 (a light coral), #C1D9B7 (a pale green), #2C7721 (a deep brown), and #00E091 (a soft mint). This palette is idea

### Prints list of Hex codes gathered from the replies from the LLM

In [21]:
print(hex_codes_list)

['#3E004A', '#ADE292', '#4120A9', '#7800A2', '#5558D8', '#D4BBDD', '#FBD3DA', '#8ADFE3', '#13292A', '#91CABE', '#499FA4', '#B5DFD6', '#E5A298', '#C1D9B7', '#2C7721', '#00E091']


### Prints list of recognized colour palettes gathered from the replies from the LLM

In [22]:
print(colour_palette_list)

['Blush of Trees', 'Lilac Shimmer', 'Candy Style', 'Fresh Succulent']


---

# Limitations

- Inability to accurately detect complex patterns or identify products with multiple colors.
- Recommendation System is computationally expensive because of the amount of data is in the image per product.
- There are no controls to set the tone/temperature.
- Limitations exist in defining the context's scope and boundaries. Occasionally, the LLM might supplement its responses with additional information.
- Catalogue dataset is not clean. When scraping, there are a lot of images that are not images of the product.

---

# Recommendations

1. More Resources for Data Collection and Cleaning:

   - Expand the dataset by collecting more diverse examples of materials and products. Gathering images that showcase a variety of material types and products with complex patterns and multi-coloured designs to improve the system’s recognition and classification capabilities.
   - Replace the incorrect images of the products to the proper images of the product to allow the recommendation system to recommend.

2. Implement Advanced Image Recognition Technologies:
   Integrate more sophisticated image processing algorithms that can handle complex patterns and multi-color detection more effectively. Consider using deep learning models trained specifically for material recognition and pattern analysis.

3. Explore Retrieval-Augmented Generation (RAG) for LLM:
   Investigate the potential of RAG to enhance the LLM's performance. By integrating a retrieval mechanism, the LLM can access a broader knowledge base during generation, allowing for more contextually relevant and updated responses. This approach would be particularly beneficial in cases where dynamic data and evolving trends influence design decisions.

---

# Conclusions

By automating the task of matching and suggesting interior design products, the system not only saves time but also introduces a level of precision in aligning with client preferences that manual processes cannot easily achieve.

However, recognizing the limitations in pattern recognition and material classification, it is clear that continued improvements and updates are necessary to maintain and enhance the system's effectiveness. By implementing the recommended actions, we can ensure that the system evolves in line with advancements in technology and changes in design trends, thereby providing enduring value to interior designers and their clients.

Through these efforts, we will support interior designers in overcoming the initial difficulties and efficiently navigating client preferences, ultimately leading to more inspired and harmonious design solutions.